In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
PATH_TRAIN_X = "../UCI_HAR_Dataset/train/X_train.txt"
PATH_TRAIN_Y = "../UCI_HAR_Dataset/train/y_train.txt"

PATH_TEST_X = "../UCI_HAR_Dataset/test/X_test.txt"
PATH_TEST_Y = "../UCI_HAR_Dataset/test/y_test.txt"

PATH_TRAIN_SBJ = "../UCI_HAR_Dataset/train/subject_train.txt"

PATH_TEST_SBJ = "../UCI_HAR_Dataset/test/subject_test.txt"

PATH_FT = "../UCI_HAR_Dataset/features.txt"
features = pd.read_csv(PATH_FT, sep=" ", header=None, index_col=0).reset_index()

In [ ]:
# training data
df_x_train = pd.read_fwf(PATH_TRAIN_X, header=None)
df_x_train.rename(columns=features[1], inplace=True)

y_train_col = pd.read_fwf(PATH_TRAIN_Y, header=None)

In [ ]:
# testing data
df_X_test = pd.read_fwf(PATH_TEST_X, header=None)
df_X_test.rename(columns=features[1], inplace=True)

df_y_test = pd.read_fwf(PATH_TEST_Y, header=None)

In [ ]:
# X = np.array(df_X_test)
X = df_X_test.to_numpy()
# X = np.random.rand(7000, 500)  # Replace with your dataset
# y = np.array(df_y_test)
y = df_y_test.to_numpy()
y = tf.keras.utils.to_categorical(y, num_classes=7)
# y = np.random.randint(0, 6, 7000)  # Replace with your dataset labels
print(f"shape-y: " + str(y.shape) + " shape-x: " + str(X.shape))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# model = tf.keras.Sequential([
#     layers.Dense(512, activation='linear', input_shape=(X_train.shape[1],)),
#     # layers.Dropout(0.5),
#     # layers.Dense(256, activation='linear'),
#     # layers.Dropout(0.5),
#     # layers.Dense(128, activation='linear'),
#     # layers.Dense(6, activation='softmax')
# ])
# model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

model = tf.keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')
])
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [8]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2, callbacks=callbacks)

In [ ]:
# model.fit(X_train, y_train, epochs=3, batch_size=8, validation_split=0.2)
model.fit(X_train, y_train, epochs=3, batch_size=8)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
x_test_final = np.array(pd.read_fwf(PATH_TEST_X, header=None))
y_test_final = np.array(pd.read_fwf(PATH_TEST_Y, header=None))

model.fit(x_test_final, y_test_final, epochs=10, batch_size=32, validation_split=0.2)

### Evaluation

`model.evaluate(x_test,  y_test, verbose=2)`

`probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])`

`probability_model(x_test[:5])`